<a href="https://colab.research.google.com/github/shuv50/IBM_Data_Science/blob/main/SpaceX_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories.

Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

# Objectives

TASK 1: Mark all launch sites on a map<br>
TASK 2: Mark the success/failed launches for each site on the map<br>
TASK 3: Calculate the distances between a launch site to its proximities

# Install Packages

In [1]:
# Install packages
!pip install folium
!pip install nest_asyncio

# Import Libraries

In [16]:
# Import libraries
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import io
import httpx
from math import sin, cos, sqrt, atan2, radians

# Collect SpaceX Data

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site.

In [3]:
# Async fetch data
async def fetch_data(url):
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        if response.status_code == 200:
            return response.content  # Return binary content
        else:
            raise Exception(f"Failed to fetch data: {response.status_code}")

In [4]:
# SpaceX data
spacex_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
data = await fetch_data(spacex_url)
spacex_df = pd.read_csv(io.BytesIO(data))
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


# Folium Maps

## Coordinates for each site

Now, we can take a look at what are the coordinates for each site

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


## Initial center location

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites.

If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. <br>Let's visualize those locations by pinning them on a map.

We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

## Folium Circle

We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate. For example,

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and we should find a small yellow circle near the city of Houston and we can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame launch_sites

In [14]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site']))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

Now, we can explore the map by zoom-in/out the marked areas , and try to answer the following questions:

Are all launch sites in proximity to the Equator line?
Are all launch sites in very close proximity to the coast?

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not

In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.







Let's first create a MarkerCluster object

In [10]:
marker_cluster = MarkerCluster()

In [15]:
# Add marker color based on class
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Create and add markers to the marker cluster
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)
site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [12]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [13]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [17]:
# Example coordinates for demonstration (replace with actual coastline coordinates obtained from MousePosition)
coastline_lat = 28.56367  # Example latitude
coastline_lon = -80.57163  # Example longitude

# Example launch site coordinates (replace with the desired launch site)
launch_site_lat = 28.561857
launch_site_lon = -80.577366

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a marker for the closest coastline point
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

# Add the marker to the map
site_map.add_child(distance_marker)

# Display the map
site_map

In [18]:
# Create a list of coordinates for the PolyLine
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Create a folium.PolyLine object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [20]:
import folium

# Example coordinates for the closest city
closest_city_coords = (launch_site_lat, launch_site_lon)

# Create a map centered around the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add a marker for the closest city
folium.Marker(
    location=closest_city_coords,
    popup='Closest City',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(m)

Based on the provided map and the ability to mark points of interest using MousePosition,
 we can visually inspect the distances and answer the questions.  Since the code provided
does not include automated distance calculations to pre-defined features like railways or highways,
the following answers are based on visual inspection and are therefore subjective.

 To make these answers more objective, we would need to incorporate datasets for railways, highways,
coastlines, and cities and calculate distances programmatically.

Are launch sites in close proximity to railways?
Answer:  Visual inspection suggests some launch sites are near railways, but not all of them.  
         Further investigation and distance calculation is required for a definitive answer.

Are launch sites in close proximity to highways?
Answer:  Many launch sites appear to be relatively close to major highways.  Again, quantitative
         distance analysis is needed for confirmation.

Are launch sites in close proximity to coastline?
Answer: Some launch sites are situated near coastlines, while others are further inland.  
         The provided code already calculates and displays the distance from one example launch site
         to a point identified as coastline. This should be replicated for all sites.

Do launch sites keep certain distance away from cities?
Answer:  It's difficult to definitively say from the current visualization. While some sites appear distant from large population centers, others seem relatively near.  
         Overlaying city boundaries and calculating distances would be necessary for a proper assessment.

Explanation of Findings:

The visual analysis suggests that proximity to transportation infrastructure (highways and potentially railways)
might be a factor in launch site selection. Coastal locations may also be favored for some launches, but not all.
The relationship to cities is unclear without detailed analysis.

To improve the analysis:
1. **Obtain Geographic Data:** Acquire datasets for railways, highways, coastlines, and city boundaries.  These datasets can be found through government agencies or organizations specializing in geographic data (e.g., OpenStreetMap).
2. **Calculate Distances Programmatically:**  Extend the provided `calculate_distance` function to compare launch sites with points from these datasets.  For coastlines, this involves determining the nearest coastline point.
3. **Statistical Analysis:**  Once distances are computed, use statistical analysis to understand correlations between launch site proximity to these features and launch success.
4. **Visual Enhancement:**  Overlay the geographic data on the map to visually inspect proximities and potentially highlight sites with specific characteristics.
